# Supervised synthetic data validation

### Importing necessary libraries

In [1]:
import numpy as np
from library.generators.NextConvGeN import NextConvGeN
from fdc.fdc import feature_clustering, canberra_modified, Clustering, FDC
from fdc.clustering import *
import pandas as pd

2023-05-09 14:10:51.382457: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-09 14:10:51.382474: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Importing data

In [2]:
data=pd.read_csv('preprocessed_cirrhosis.csv')

In [3]:
data=data.rename(columns = {'Stage': 'Target'})

In [4]:
np.random.seed(42)
data=data.sample(frac=1)

In [5]:
values=['N_Days','Age','Bilirubin','Cholesterol','Albumin','Copper','Alk_Phos',
        'SGOT','Tryglicerides','Platelets','Prothrombin','Ascites','Hepatomegaly','Spiders','Edema',
       'Status','Drug','Sex','Target']

In [6]:
data=data[values]

### Train-val split

In [7]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(np.array(data.drop(["Target"],axis=1)), np.array(data['Target']), test_size = 0.25, random_state = 42)

### Gradient boosting classifier for original training data (Model-1)

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
np.random.seed(42)
model_1 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=2, random_state=42)
model_1.fit(train_features,  train_labels)

GradientBoostingClassifier(learning_rate=0.5, max_depth=2, max_features=2,
                           n_estimators=20, random_state=42)

In [9]:
from sklearn.model_selection import cross_val_score
np.random.seed(42)
score_lr=cross_val_score(model_1, train_features, train_labels,cv=5)
print(score_lr)
print("Avg :",np.average(score_lr))

[0.52380952 0.3968254  0.41269841 0.48387097 0.41935484]
Avg : 0.4473118279569893


### Preparing training data for synthetic data generation

In [10]:
train_features=pd.DataFrame(train_features)

In [11]:
train_features['Target']=train_labels

In [12]:
train_features.to_csv('Liver_cirrhosis_training_data(sup).csv',index=False)

In [13]:
hold_out_data=pd.DataFrame(test_features)
hold_out_data['Target']=test_labels
hold_out_data.columns=values
hold_out_data.to_csv('Liver_cirrhosis_hold_out__data(sup).csv',index=False)

In [14]:
dicts = {}
for i in range(len(values)):
        dicts[values[i]]=i


In [15]:
train_features.rename(columns=dicts,inplace=True)


In [16]:
train_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1375.0,22646.0,7.3,246.0,3.49,24.0,797.000000,121.175000,113.0,189.0,10.9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0
1,3395.0,19295.0,3.2,259.0,4.30,208.0,1040.000000,110.050000,78.0,268.0,11.7,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0
2,785.0,19358.0,12.6,196.0,2.87,73.0,289.000000,145.333333,68.0,114.0,11.8,0.0,1.0,0.0,0.0,1.0,0.0,0.0,4.0
3,786.0,16839.0,2.9,332.0,3.60,86.0,1492.000000,134.850000,103.0,277.0,11.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,4.0
4,4509.0,23331.0,0.7,370.0,3.78,24.0,5833.000000,73.530000,86.0,390.0,10.6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2574.0,19237.0,0.5,251.0,4.52,31.0,784.000000,74.400000,58.0,361.0,10.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
309,1444.0,19002.0,5.2,1128.0,3.68,53.0,3228.000000,165.850000,166.0,421.0,9.9,0.0,1.0,1.0,0.0,1.0,1.0,0.0,3.0
310,2716.0,19358.0,0.6,251.0,4.19,25.0,1435.033333,71.121667,58.0,330.0,9.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
311,3458.0,20600.0,0.6,172.0,4.64,20.0,666.000000,54.250000,69.0,265.0,10.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


### FDC for searcing neighbors for synthetic data generation

In [17]:
from fdc.fdc import feature_clustering, canberra_modified, Clustering, FDC
np.random.seed(42)
fdc = FDC()
fdc.ord_list=list(range(data.shape[1]))[11:15]
fdc.nom_list = list(range(data.shape[1]))[15:]
fdc.cont_list =list(range(data.shape[1]))[:11]

In [18]:
train_features=np.array(train_features)

### Preapring and training NextConvGen model

In [19]:
gen = NextConvGeN(train_features.shape[1], neb=5, fdc=fdc)

In [20]:
gen.reset(train_features)

2023-05-09 14:10:56.348322: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-09 14:10:56.348362: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-09 14:10:56.348379: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (f61d203ff696): /proc/driver/nvidia/version does not exist
2023-05-09 14:10:56.348533: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
train_features

array([[1.3750e+03, 2.2646e+04, 7.3000e+00, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+00],
       [3.3950e+03, 1.9295e+04, 3.2000e+00, ..., 1.0000e+00, 1.0000e+00,
        3.0000e+00],
       [7.8500e+02, 1.9358e+04, 1.2600e+01, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+00],
       ...,
       [2.7160e+03, 1.9358e+04, 6.0000e-01, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [3.4580e+03, 2.0600e+04, 6.0000e-01, ..., 0.0000e+00, 0.0000e+00,
        2.0000e+00],
       [1.2950e+03, 1.6513e+04, 1.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        4.0000e+00]])

In [22]:
from fdc.fdc import feature_clustering, canberra_modified, Clustering, FDC
np.random.seed(42)
gen.train(train_features)

FDC.normalize (init): 0.00002 / 0.000s
FDC.normalize (clustering CONT): 6.65731 / 6.657s
FDC.normalize (clustering ORD): 3.90395 / 10.561s
FDC.normalize (clustering NOM): 3.33580 / 13.897s
FDC.normalize (concat): 0.00007 / 13.897s
FDC.normalize (total): 0.00001 / 13.897s
|N| = (313, 5)
|D| = (313, 19)


KeyboardInterrupt: 

### Generating 1000 synthetic points and randomly selecting 300 points for evaluation

In [ ]:
from fdc.NN_models import *
np.random.seed(42)
syn_train_data = syn_data_gen(gen,1000,values)

In [ ]:
def balanced_synthetic_data(org_data,syn_data):
    import math
    org_data_len = org_data.shape[0]
    class_counts = org_data.iloc[:,-1].value_counts()
    column_names = list( syn_data.columns)
    df_list=[]
    for i in range(len(class_counts)):
        m = len(syn_data[syn_data['Target']==i])

        if m < class_counts[i]:
            run=math.ceil(class_counts[i]/m)
            syn_data = syn_data_gen(gen,org_data_len*run,values)
        c_i=syn_data[syn_data['Target']==i].sample(n=int(class_counts[i]),axis=0,random_state=42)
        df_list.append(c_i)
    balanced_df=pd.concat(df_list,names=column_names)
    return balanced_df

In [ ]:
syn_train_data=balanced_synthetic_data(pd.DataFrame(train_features),syn_train_data)

In [ ]:
syn_train_data.to_csv('Liver_cirrhosis_NextConvGen_syn_data(supervised).csv',index=False)

In [ ]:
syn_train_data['Target'].value_counts()

In [ ]:
np.random.seed(42)
syn_train_data=syn_train_data.sample(frac=1) #Shuffle the data set
np.random.seed(42)
i=[x for x in range(np.shape(syn_train_data)[0])]

syn_train_data.set_index(pd.Series(i), inplace=True)

In [ ]:
def exact_match_score(original_df, synthetic_df):
    original_data=original_df.drop('Target',axis=1,inplace=False)
    synthetic_data=synthetic_df.drop('Target',axis=1,inplace=False)
    n_matches = 0
    repeated=[]
    for i in range(synthetic_data.shape[0]):
        if (original_data == synthetic_data.iloc[i, :]).all(axis=1).any():
            repeated_data=synthetic_data.iloc[i, :]
            repeated.append(repeated_data)
            n_matches += 1    
    print('The number of data points repeated in the synthetic data are',n_matches / synthetic_data.shape[0])
    return pd.DataFrame(repeated)

In [ ]:
original_df=pd.read_csv('Liver_cirrhosis_training_data(sup).csv')
original_df.columns=values

In [ ]:
exact_match_score(original_df,syn_train_data)

In [ ]:
len(syn_train_data[syn_train_data.duplicated(keep=False)])

### Train test split on synthetic data

In [ ]:
syn_train_features, syn_test_features, syn_train_labels, syn_test_labels = train_test_split(np.array(syn_train_data.drop(['Target'],axis=1)), np.array(syn_train_data['Target']), test_size = 0.25, random_state = 42)

### Gradient boosting classifier for synthetic samples traing data (Model-2)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
NextConvGen_model_2 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=2, random_state=42)


### 5 Fold Cross Validation

In [ ]:
score_lr=cross_val_score(NextConvGen_model_2, syn_train_features, syn_train_labels,cv=5)
print(score_lr)
print("Avg :",np.average(score_lr))


In [ ]:
NextConvGen_model_2.fit(syn_train_features,syn_train_labels )

### Case-1: predicting validation data using Model-1


In [ ]:
x=model_1.predict(np.array(test_features))
cluster_wise_F1score(list(test_labels),list(x))

### Case-2: Predicting validation data using Model-2


In [ ]:
y=NextConvGen_model_2.predict(np.array(test_features))

cluster_wise_F1score(list(test_labels),list(y))

### Comparing predicted values from Case-1 & Case-2

In [ ]:
cluster_wise_F1score(list(x),list(y))

### CTGAN

In [ ]:
train_features=pd.DataFrame(train_features)

In [ ]:
dicts = {}
for i in range(len(values)):
        dicts[i]=values[i]

In [ ]:
train_features.rename(columns=dicts,inplace=True)


In [ ]:
train_features

In [ ]:
from sdv.tabular import CTGAN

In [ ]:
gen=CTGAN()

In [ ]:
gen.fit(train_features)

### Generating 1000 synthetic points and randomly selecting 300 points for evaluation

In [ ]:
CTGAN_syn_data= gen.sample(num_rows=462)

In [ ]:
from fdc.NN_models import *
np.random.seed(42)
CTGAN_syn_data=balanced_synthetic_data_CTGAN(train_features,CTGAN_syn_data,gen)

In [ ]:
CTGAN_syn_data.to_csv('Liver_cirrhosis_CTGAN_syn_data(supervised).csv',index=False)

In [ ]:
np.random.seed(42)
CTGAN_syn_data=CTGAN_syn_data.sample(frac=1) #Shuffle the data set
np.random.seed(42)
i=[x for x in range(np.shape(CTGAN_syn_data)[0])]

CTGAN_syn_data.set_index(pd.Series(i), inplace=True)

### Train test split on synthetic data

In [ ]:
syn_train_features, syn_test_features, syn_train_labels, syn_test_labels = train_test_split(np.array(CTGAN_syn_data.drop(["Target"],axis=1)), np.array(CTGAN_syn_data["Target"]), test_size = 0.25, random_state = 42)

### Gradient boosting classifier on CTGAN generated synthetic training data  (Model-2)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
CTGAN_model_2 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=2, random_state=42)


### 5 Fold Cross validation

In [ ]:
score_lr=cross_val_score(CTGAN_model_2, syn_train_features, syn_train_labels,cv=5)
print(score_lr)
print("Avg :",np.average(score_lr))


In [ ]:
CTGAN_model_2.fit(syn_train_features,syn_train_labels )

### Case-1: predicting validation data using Model-1


In [ ]:
x=model_1.predict(np.array(test_features))
cluster_wise_F1score(list(test_labels),list(x))

### Case-2: predicting validation data using Model-2


In [ ]:
y=CTGAN_model_2.predict(np.array(test_features))

cluster_wise_F1score(list(test_labels),list(y))

### Comparing predicted values from Case-1 & Case-2

In [ ]:
cluster_wise_F1score(list(x),list(y))

### CTABGAN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from library.generators.ctab import CtabGan

In [ ]:
gen=CtabGan()

In [ ]:
gen.reset(train_features)

In [ ]:
gen.train(train_features)

### Generating 1000 synthetic points and randomly selecting 300 points for evaluation

In [ ]:
CTABGAN_syn_data= gen.generateData(1000)

In [ ]:
CTABGAN_syn_data=pd.DataFrame(CTABGAN_syn_data)

In [ ]:
CTABGAN_syn_data.rename(columns=dicts,inplace=True)


In [ ]:
CTABGAN_syn_data

In [ ]:
for i in values[7:]:
    CTABGAN_syn_data[i]=abs(np.round(CTABGAN_syn_data[i]))

In [ ]:
from fdc.NN_models import *
np.random.seed(42)
CTABGAN_syn_data=balanced_synthetic_data_CTABGAN(train_features,CTABGAN_syn_data,gen)

In [ ]:
CTABGAN_syn_data.to_csv('Liver_cirrhosis_CTABGAN_syn_data(supervised).csv',index=False)

In [ ]:
np.random.seed(42)
CTABGAN_syn_data=CTABGAN_syn_data.sample(frac=1) #Shuffle the data set
np.random.seed(42)
i=[x for x in range(np.shape(CTABGAN_syn_data)[0])]

CTABGAN_syn_data.set_index(pd.Series(i), inplace=True)

### Train test split on synthetic data

In [ ]:
syn_train_features, syn_test_features, syn_train_labels, syn_test_labels = train_test_split(np.array(CTABGAN_syn_data.drop(["Target"],axis=1)), np.array(CTABGAN_syn_data["Target"]), test_size = 0.25, random_state = 42)

### Gradient boosting classifier on CTGAN generated synthetic training data  (Model-2)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
CTABGAN_model_2 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=2, random_state=42)


### 5 Fold Cross validation

In [ ]:
score_lr=cross_val_score(CTABGAN_model_2, syn_train_features, syn_train_labels,cv=5)
print(score_lr)
print("Avg :",np.average(score_lr))


In [ ]:
CTABGAN_model_2.fit(syn_train_features,syn_train_labels )

### Case-1: predicting validation data using Model-1


In [ ]:
x=model_1.predict(np.array(test_features))
cluster_wise_F1score(list(test_labels),list(x))

### Case-2: predicting validation data using Model-2


In [ ]:
y=CTABGAN_model_2.predict(np.array(test_features))

cluster_wise_F1score(list(test_labels),list(y))

### Comparing predicted values from Case-1 & Case-2

In [ ]:
cluster_wise_F1score(list(x),list(y))